# get_pond_properties
Author: Connor Finn <br>
Date: July 29, 2020 <br>
Description: <dir>
    This notebook will be used to determine the geometric properties of the salton sea.

In [1]:
import ee
import numpy as np
from IPython.display import Image

# Trigger the authentication flow.
ee.Authenticate()

Enter verification code: 4/2QHUKRmDQVVI4Hx0PHeDuUBp8EaxwvBCtVgb_ruEUkbFwMyXdBJxWHs

Successfully saved authorization token.


In [2]:
ee.Initialize()

# Collect an image

## Part 1: Collect the images - all during the month of May (same as time lapse)

In [50]:
def get_images(path_list , row_list , satelite , start_date , end_date , max_cloud_percentage , months):
    
    # get image collection object
    coll = ee.ImageCollection(satelite)\
        .filterDate(start_date, end_date)\
        .filter(ee.Filter.inList('WRS_PATH', path_list))\
        .filter(ee.Filter.inList('WRS_ROW', row_list))\
        .filter(ee.Filter.lt('CLOUD_COVER' , max_cloud_percentage))\
        .filter(ee.Filter.calendarRange(months[0],months[1],'month'))    # just may data
    # get image_id's
    image_ids = list( map( lambda x : x['id'] , coll.getInfo()['features'] ) ) 
    
    # get image objects
    images = list( map( lambda x: ee.Image(x) , image_ids ) )
    
    return images


In [51]:
# fill in the information
p = [39]                                    # path       
r = [37]                                    # row 
sat = 'LANDSAT/LC08/C01/T1_TOA'             # use this satelite
sd = '2013-05-01'                           # start date
ed = '2020-05-01'                           # end date
cc= 15                                      # max cloud cover percentage
months = [5 , 5]                            # months [start, end]
image_list = get_images(p, r, sat, sd, ed, cc , months)

## Reduce the images to just the lake

In [52]:
lake_bounding_box = ([-116.21,33.58],\
 [-116.19,33.03],\
 [-115.36,33.04],\
 [-115.40,33.65])

lake_region = ee.Geometry.Rectangle([-116.21 , 33.03 , -115.36 , 33.65])

In [53]:
# This function adds a band with the ndvi.

def addNDWI(image):
    return image.addBands(image.normalizedDifference(['B5', 'B4']))


def addNDVI(image):
    return image.addBands(image.normalizedDifference(['B5', 'B3']))

In [54]:
# this function clip to the lake
def clip(image):
    lake_region = ee.Geometry.Rectangle([-116.21 , 33.03 , -115.36 , 33.65])
    return image.clip(lake_region)

## Add NDWI, and display

In [55]:
image_list = list(map(addNDWI, image_list))        # set as 'nd'
image_list = list(map(clip, image_list))


parameters = {'min': -1.0, # value which is mapped to 0
              'max': 1.0,  # value which is mapped to 255
              'dimensions': 768,   # size of the image 
              'bands': ['nd'],     # The bands we select
              'palette': ['blue', 'white', 'green'],  # only use a pallete when visualizing one.
              #'region': lake_region
}

lake_image = image_list[0]
display(Image(url = lake_image.getThumbUrl(parameters)))

## Add NDVI, and display

In [56]:
image_list = list(map(addNDVI, image_list))        # set as 'nd_1'
parameters = {'min': -1.0, # value which is mapped to 0
              'max': 1.0,  # value which is mapped to 255
              'dimensions': 768,   # size of the image 
              'bands': ['nd_1'],     # The bands we select
              'palette': ['blue', 'white', 'green'],  # only use a pallete when visualizing one.
              #'region': lake_region
}

lake_image = image_list[0]
display(Image(url = lake_image.getThumbUrl(parameters)))

In [61]:
lake_image.getInfo()['bands']

[{'id': 'B1',
  'data_type': {'type': 'PixelType', 'precision': 'float'},
  'dimensions': [2665, 2323],
  'origin': [1968, 1878],
  'crs': 'EPSG:32611',
  'crs_transform': [30, 0, 514185, 0, -30, 3780915]},
 {'id': 'B2',
  'data_type': {'type': 'PixelType', 'precision': 'float'},
  'dimensions': [2665, 2323],
  'origin': [1968, 1878],
  'crs': 'EPSG:32611',
  'crs_transform': [30, 0, 514185, 0, -30, 3780915]},
 {'id': 'B3',
  'data_type': {'type': 'PixelType', 'precision': 'float'},
  'dimensions': [2665, 2323],
  'origin': [1968, 1878],
  'crs': 'EPSG:32611',
  'crs_transform': [30, 0, 514185, 0, -30, 3780915]},
 {'id': 'B4',
  'data_type': {'type': 'PixelType', 'precision': 'float'},
  'dimensions': [2665, 2323],
  'origin': [1968, 1878],
  'crs': 'EPSG:32611',
  'crs_transform': [30, 0, 514185, 0, -30, 3780915]},
 {'id': 'B5',
  'data_type': {'type': 'PixelType', 'precision': 'float'},
  'dimensions': [2665, 2323],
  'origin': [1968, 1878],
  'crs': 'EPSG:32611',
  'crs_transform': 